In [1]:
import os
import sys
import timeit

import h5py
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams

sys.path.append('../')
sys.path.append('../deep_learning/')

from deep_learning.classes_and_functions import Linear_SdA
from deep_learning.logistic_sgd import load_data
from deep_learning.utils import tile_raster_images
from deep_learning.dA import dA
from deep_learning.SdA import SdA

try:
    import PIL.Image as Image
except ImportError:
    import Image


pretrain_lr=0.1
pretrain_lr_binary = 1e-2

batch_size=128

with h5py.File('../../deep_learning_files/flickr_32x32.hdf5') as f:
    X = f['X'][:] 

X_std = np.std(X)
X_mean = np.mean(X)
X = (X-X_mean) / X_std

train_set_x = X[:24000] 
valid_set_x = X[24000:24500]
test_set_x = X[24500:25000]



train_set_x = theano.shared(np.asarray(train_set_x,
                                               dtype=theano.config.floatX),
                                 borrow=True)

valid_set_x = theano.shared(np.asarray(valid_set_x,
                                               dtype=theano.config.floatX),
                                 borrow=True)

test_set_x = theano.shared(np.asarray(test_set_x,
                                               dtype=theano.config.floatX),
                                 borrow=True)

datasets = (train_set_x, valid_set_x, test_set_x)

# compute number of minibatches for training, validation and testing
n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size

numpy_rng = np.random.RandomState(89677)
print ('... building the model')
# construct the stacked denoising autoencoder class
sda = Linear_SdA(
    numpy_rng=numpy_rng,
    n_ins=32*32*3,
    hidden_layers_sizes=[8192, 4096, 2048, 1024, 512, 256]
)

... building the model


In [ ]:
pretraining_epochs = 500
pretrain_lr=0.1

print ('... getting the pretraining functions')
pretraining_fns = sda.pretraining_functions(train_set_x=train_set_x,
                                            batch_size=batch_size)
print ('... pre-training the linear-binary layer')
start_time = timeit.default_timer()
## Pre-train layer-wise
corruption_levels = [.3, .2, .1, .05, .025, .01]
for i in range(0,1):
    # go through pretraining epochs
    for epoch in range(pretraining_epochs):
        # go through the training set
        c = []
        for batch_index in range(n_train_batches):
            c.append(pretraining_fns[i](index=batch_index,
                     corruption=corruption_levels[i],
                     lr=pretrain_lr))
        print('Pre-training layer %i, epoch %d, cost %f' % (i, epoch, np.mean(c) ))

... getting the pretraining functions


../deep_learning/classes_and_functions.py:303: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)
  theano.Param(corruption_level, default=0.2),
../deep_learning/classes_and_functions.py:304: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)
  theano.Param(learning_rate, default=0.1)


... pre-training the linear-binary layer
Pre-training layer 0, epoch 0, cost 2.167951
Pre-training layer 0, epoch 1, cost 2.102736
Pre-training layer 0, epoch 2, cost 2.059220
Pre-training layer 0, epoch 3, cost 2.025874
Pre-training layer 0, epoch 4, cost 1.999170
Pre-training layer 0, epoch 5, cost 1.976729
Pre-training layer 0, epoch 6, cost 1.956926
Pre-training layer 0, epoch 7, cost 1.939409
Pre-training layer 0, epoch 8, cost 1.923809


In [ ]:
#pretrain binary layers
print ('... pre-training the binary-binary layers')
pretraining_epochs = 300
pretrain_lr_binary = 1e-2

for i in range(1,sda.n_layers):
    # go through pretraining epochs
    for epoch in range(pretraining_epochs):
        # go through the training set
        c = []
        for batch_index in range(n_train_batches):
            c.append(pretraining_fns[i](index=batch_index,
                     corruption=corruption_levels[i],
                     lr=pretrain_lr_binary))
        print('Pre-training layer %i, epoch %d, cost %f' % (i, epoch, np.mean(c) ))

In [ ]:
# FINETUNING THE MODEL #
########################
finetune_lr=1e-4
training_epochs=1000
patience = 1000 * n_train_batches  # look as this many examples regardless
patience_increase = 2.  # wait this much longer when a new best is
                        # found
improvement_threshold = 0.995  # a relative improvement of this much is
                               # considered significant
    

# get the training, validation and testing function for the model
print ('... getting the finetuning functions')
train_fn, validate_model, test_model = sda.build_finetune_functions(
    datasets=datasets,
    batch_size=batch_size,
    learning_rate=finetune_lr
)

print ('... finetunning the model')
# early-stopping parameters

validation_frequency = min(n_train_batches, patience / 2)
                              # go through this many
                              # minibatche before checking the network
                              # on the validation set; in this case we
                              # check every epoch

best_validation_loss = numpy.inf
test_score = 0.
start_time = timeit.default_timer()

done_looping = False
epoch = 0

while (epoch < training_epochs) and (not done_looping):
    epoch = epoch + 1
    for minibatch_index in range(n_train_batches):
        minibatch_avg_cost = train_fn(minibatch_index)
        iter = (epoch - 1) * n_train_batches + minibatch_index

        if (iter + 1) % validation_frequency == 0:
            validation_losses = validate_model()
            this_validation_loss = np.mean(validation_losses)
            print_and_log('epoch %i, minibatch %i/%i, validation RMSE of%f' %
                  (epoch, minibatch_index + 1, n_train_batches,
                   this_validation_loss ))

            # if we got the best validation score until now
            if this_validation_loss < best_validation_loss:

                #improve patience if loss improvement is good enough
                if (
                    this_validation_loss < best_validation_loss *
                    improvement_threshold
                ):
                    patience = max(patience, iter * patience_increase)

                # save best validation score and iteration number
                best_validation_loss = this_validation_loss
                best_iter = iter

                # test it on the test set
                test_losses = test_model()
                test_score = np.mean(test_losses)
                print(('     epoch %i, minibatch %i/%i, test RMSE of '
                       'best model %f') %
                      (epoch, minibatch_index + 1, n_train_batches,
                       test_score ))

        if patience <= iter:
            done_looping = True
            break